In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE


In [2]:
dataset = pd.read_csv(r"C:\Users\NSS3112\Desktop\ml\malware trainnig notebook\datasetandroidpermissions\dataset.csv")

dataset.shape
print(len(dataset))
print(len(dataset[dataset.type==0]))
print(len(dataset[dataset.type==1]))

398
199
199


In [3]:
dataset.iloc[50:60,76:]

,android.permission.CALL_PHONE,android.permission.CALL_PRIVILEGED,android.permission.CAMERA,android.permission.CAMERA_DISABLE_TRANSMIT_LED,android.permission.CAPTURE_AUDIO_HOTWORD,android.permission.CAPTURE_AUDIO_OUTPUT,android.permission.CAPTURE_SECURE_VIDEO_OUTPUT,android.permission.CAPTURE_TV_INPUT,android.permission.CAPTURE_VIDEO_OUTPUT,android.permission.CARRIER_FILTER_SMS,...,com.android.voicemail.permission.WRITE_VOICEMAIL,com.foo.mypermission,com.foo.mypermission2,org.chromium.chrome.shell.permission.C2D_MESSAGE,org.chromium.chrome.shell.permission.DEBUG,org.chromium.chrome.shell.permission.SANDBOX,org.chromium.chromecast.shell.permission.SANDBOX,org.chromium.content_shell.permission.SANDBOX,test_permission,type
50,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
51,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
52,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
54,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
55,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
57,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
59,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
x_train,x_test, y_train, y_test = train_test_split(dataset.drop(columns={"type"}),dataset.type,train_size=0.8)
X = dataset.drop(columns={"type"})
y = dataset.type

In [10]:
#fonction calucule de moyenne pour validaton croisée
def moyenne(entree):
    s = 0
    for i in entree:
        s = s + i
    return s/len(entree)

In [11]:
#bagging

In [12]:
 from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [13]:
model = BaggingClassifier(base_estimator=KNeighborsClassifier(),
                         n_estimators=100)

model.fit(x_train, y_train)
print("l'accuracy est de:")
model.score(x_test, y_test)

l'accuracy est de:


0.9125

In [14]:
#boosting

In [15]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

In [19]:
model = AdaBoostClassifier(n_estimators=100)
model.fit(x_train, y_train)
print("l'accuracy est de:")
model.score(x_test, y_test)


l'accuracy est de:


0.8875

In [18]:
#stacking

In [20]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [23]:
model_1 = SGDClassifier(random_state=0)
model_2 = DecisionTreeClassifier(random_state=0)
model_3 = KNeighborsClassifier(n_neighbors=2)

model_4 = VotingClassifier([('SGD', model_1),
                            ('Tree', model_2),
                            ('KNN', model_3)],
                          voting='hard')

for model in (model_1, model_2, model_3, model_4):
    model.fit(x_train, y_train)
    print(model.__class__.__name__, model.score(x_test, y_test))

SGDClassifier 0.9375
DecisionTreeClassifier 0.9375
KNeighborsClassifier 0.9125
VotingClassifier 0.95


In [24]:
model = StackingClassifier([('SGD', model_1),
                            ('Tree', model_2),
                            ('KNN', model_3)],
                             final_estimator=KNeighborsClassifier())

model.fit(x_train, y_train)
print("l'accuracy est de:")
model.score(x_test, y_test)


l'accuracy est de:


0.9375

In [26]:
#si valeur egal à 0 on l'elimine 
for attribut in X.columns:
    if X[attribut].var() == 0:
        X = X.drop(columns={attribut})

In [27]:
#La selction d'attributs
#nombre d'attributs
nof_list=np.arange(10,93)            
high_score=0
#pour sauvgarder l'optimal
nof=0           
score_list =[]
for n in range(len(nof_list)):
    model = LogisticRegression(solver="lbfgs",penalty="l2",max_iter=250)
    rfe = RFE(model,nof_list[n])
    X_rfe = rfe.fit_transform(X,y)
    score = moyenne(cross_val_score(model,X_rfe,y,cv=5))
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]


print("Le nombre oprimal est de: %d" %nof)
print("le score avec %d attributs: %f" % (nof, high_score))

C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=11 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=12 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=13 as keyword args. From version 0.25 pass

C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=40 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=41 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=42 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=43 as keyword args. From version 0.25 pass

C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=70 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=71 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=72 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=73 as keyword args. From version 0.25 pass

Le nombre oprimal est de: 23
le score avec 23 attributs: 0.947342


C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=92 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [28]:
#selection des attribut finaux
cols = list(X.columns)
rfe = RFE(model, 23)             
X_rfe = rfe.fit_transform(X,y)  
model.fit(X_rfe,y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe= temp[temp==True].index
print(selected_features_rfe)

C:\Users\NSS3112\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass n_features_to_select=23 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Index(['android.permission.BLUETOOTH', 'android.permission.CAMERA',
       'android.permission.CHANGE_WIFI_STATE',
       'android.permission.DEVICE_POWER',
       'android.permission.DISABLE_KEYGUARD', 'android.permission.GET_TASKS',
       'android.permission.INSTALL_PACKAGES', 'android.permission.INTERNET',
       'android.permission.MODIFY_AUDIO_SETTINGS',
       'android.permission.MOUNT_UNMOUNT_FILESYSTEMS',
       'android.permission.READ_CALENDAR', 'android.permission.READ_CALL_LOG',
       'android.permission.READ_PHONE_STATE', 'android.permission.READ_SMS',
       'android.permission.RECEIVE_BOOT_COMPLETED',
       'android.permission.SEND_SMS',
       'android.permission.SET_PREFERRED_APPLICATIONS',
       'android.permission.USE_CREDENTIALS', 'android.permission.WAKE_LOCK',
       'android.permission.WRITE_APN_SETTINGS',
       'android.permission.WRITE_SETTINGS', 'android.permission.WRITE_SMS',
       'com.android.browser.permission.READ_HISTORY_BOOKMARKS'],
      dtype='o

In [29]:
X = X[['android.permission.BLUETOOTH', 'android.permission.CAMERA',
       'android.permission.CHANGE_WIFI_STATE',
       'android.permission.DEVICE_POWER',
       'android.permission.DISABLE_KEYGUARD', 'android.permission.GET_TASKS',
       'android.permission.INSTALL_PACKAGES', 'android.permission.INTERNET',
       'android.permission.MODIFY_AUDIO_SETTINGS',
       'android.permission.MOUNT_UNMOUNT_FILESYSTEMS',
       'android.permission.READ_CALENDAR', 'android.permission.READ_CALL_LOG',
       'android.permission.READ_PHONE_STATE', 'android.permission.READ_SMS',
       'android.permission.RECEIVE_BOOT_COMPLETED',
       'android.permission.SEND_SMS',
       'android.permission.SET_PREFERRED_APPLICATIONS',
       'android.permission.USE_CREDENTIALS', 'android.permission.WAKE_LOCK',
       'android.permission.WRITE_APN_SETTINGS',
       'android.permission.WRITE_SETTINGS', 'android.permission.WRITE_SMS',
       'com.android.browser.permission.READ_HISTORY_BOOKMARKS']]
X

,android.permission.BLUETOOTH,android.permission.CAMERA,android.permission.CHANGE_WIFI_STATE,android.permission.DEVICE_POWER,android.permission.DISABLE_KEYGUARD,android.permission.GET_TASKS,android.permission.INSTALL_PACKAGES,android.permission.INTERNET,android.permission.MODIFY_AUDIO_SETTINGS,android.permission.MOUNT_UNMOUNT_FILESYSTEMS,...,android.permission.READ_SMS,android.permission.RECEIVE_BOOT_COMPLETED,android.permission.SEND_SMS,android.permission.SET_PREFERRED_APPLICATIONS,android.permission.USE_CREDENTIALS,android.permission.WAKE_LOCK,android.permission.WRITE_APN_SETTINGS,android.permission.WRITE_SETTINGS,android.permission.WRITE_SMS,com.android.browser.permission.READ_HISTORY_BOOKMARKS
0,0,0,0,0,1,0,0,1,0,0,...,1,1,1,0,0,1,1,0,1,0
1,0,0,0,0,0,1,0,1,0,0,...,1,0,1,0,0,0,1,0,1,0
2,0,0,0,0,1,0,0,1,0,0,...,1,1,1,0,0,1,1,0,1,0
3,0,0,0,0,1,0,0,1,0,0,...,1,1,1,0,0,1,1,0,1,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
394,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0


In [30]:
#Etrainement finale
model = RandomForestClassifier(n_estimators=100)
model.fit(X,y)

#Sérialisation
import joblib
joblib.dump(model,'model_bagging.pkl')

['model_bagging.pkl']

In [32]:
inpt_list = [0,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0]
inpt = np.array([inpt_list])
print(model.predict(inpt)[0])

1
